In [ ]:
import sys
import geopandas as gpd
from shutil import copyfile, move

sys.path.insert(0, './HLB')
import hlb_utils

### Dataset

In [ ]:
pre_processing = hlb_utils.PreProcessing('../IMGS/odm_orthophoto_2.tif', proj_name='anno')

### Split image [and vectors]

In [ ]:
pre_processing.split_img(patch_size=256, overlap=0, save_img=False)

In [ ]:
pre_processing.split_vector()

### Make annotation

In [ ]:
pre_processing.grid = gpd.read_file('../datasets/anno/img_grid.geojson').set_index('id')

In [ ]:
pre_processing.make_annotations()

### Outros

In [48]:
import os
from osgeo import gdal, osr
from PIL import Image

shape_path_aux = '../IMGS/TALHAO_ANGULADAS/INF'

result_paths = []
for filename in os.listdir(shape_path_aux):
    result_paths.append(os.path.join(shape_path_aux, filename))

for path in result_paths:
    
    img = Image.open(path)
    w, h = img.size
    img = img.crop((1000, 700, w, h))
    out_path = os.path.join(os.path.dirname(os.path.dirname(path)), 'EDITS', os.path.basename(path))
    img.save(out_path)

In [1]:
import os
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from time import sleep


shape_path_aux = '../datasets/ortofoto_talhao/results'

with tqdm(total=len(os.listdir(shape_path_aux)) as pbar:
    pbar.set_description("Lendo arquivos de entradas")
    sleep(0.1)

    result_paths = []
    for filename in os.listdir(shape_path_aux):
        if os.path.splitext(filename)[1].lower() == '.geojson':
            result_paths.append(os.path.join(shape_path_aux, filename))
        pbar.update(1)
    

pols = pd.concat([gpd.read_file(i) for i in result_paths], axis=0).reset_index(drop=True)
pols['geometry'] = pols.buffer(-0.1).buffer(0.1).simplify(0.03)
pols = pols.explode().reset_index(drop=True)
pols['centroid'] = pols['geometry'].centroid
pols.insert(1, column='union', value=0)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)

with tqdm(total=len(pols.index)) as pbar:
    pbar.set_description("Simplificando - Macro")
    sleep(0.1)
    for i in pols.index:
        if pols.loc[i, 'union'] == 0:
            point = pols.loc[i, 'centroid']
            distance = pols['centroid'].distance(point)

            distance = distance[distance <= 1]

            if len(distance) > 1:
                pols.at[distance.index, 'union'] = i
            else:
                pols.at[i, 'union'] = i
        pbar.update(1)

pols.drop('centroid', axis=1, inplace=True)
pols = pols.dissolve(by='union', as_index=False)
pols.drop(pols.loc[pols['geometry'].is_empty].index, axis=0, inplace=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols['id'] = pols.index
# pols['centroid'] = pols['geometry'].centroid


over = gpd.overlay(pols, pols, how='intersection')
over = over.loc[-over['geometry'].is_empty]
over = over.loc[over['id_1'] != over['id_2']].explode().reset_index(drop=True)
delete_lines = []
with tqdm(total=len(over['id_1'])) as pbar:
    pbar.set_description("Simplificando - Micro")
    sleep(0.1)

    for i in over['id_1']:

        intersec = over[over['id_1']==i]

        if intersec.loc[intersec['geometry'].area >= 0.6 * pols.loc[i, 'geometry'].area].any(axis=None) or \
        (3.5 * pols.loc[i, 'geometry'].area < pols.loc[intersec['id_2'], 'geometry'].area).any():
            delete_lines.append(i)
            
        pbar.update(1)

    delete_lines = (list(set(delete_lines)))

pols.drop(labels=delete_lines, axis=0, inplace=True)
pols.explode().reset_index(drop=True)
pols.set_index(keys=pd.Index(range(1, pols.shape[0] + 1)), inplace=True)
pols.to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_result.geojson'), driver='GeoJSON')

Simplificando - Micro: 100%|██████████| 6344/6344 [00:53<00:00, 119.31it/s]


In [ ]:
pols['id'] = pols.index
pols

In [ ]:
pols.to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_result_1.geojson'), driver='GeoJSON')

In [144]:
over = gpd.overlay(pols, pols, how='intersection')
over = over.loc[-over['geometry'].is_empty]
over = over.loc[over['id_1'] != over['id_2']].explode().reset_index(drop=True)

In [145]:
over

,union_1,score_1,detection_score_1,id_1,union_2,score_2,detection_score_2,id_2,geometry
0,3,None,0.977909,3,1,None,0.996637,1,"POLYGON ((294197.035 7515266.931, 294197.031 7..."
1,1,None,0.996637,1,3,None,0.977909,3,"POLYGON ((294197.030 7515266.937, 294197.032 7..."
2,3950,None,0.904703,1974,2,None,0.992981,2,"POLYGON ((294187.353 7515268.945, 294187.360 7..."
3,2,None,0.992981,2,3950,None,0.904703,1974,"POLYGON ((294185.359 7515268.850, 294185.386 7..."
4,267,None,0.999685,246,3950,None,0.904703,1974,"POLYGON ((294187.309 7515270.952, 294187.313 7..."
...,...,...,...,...,...,...,...,...,...
3597,3981,None,0.999364,1982,3946,None,0.594121,1972,"POLYGON ((294214.008 7515409.130, 294214.100 7..."
3598,3964,None,0.999864,1977,3963,None,0.999929,1976,"POLYGON ((294364.802 7515343.147, 294364.905 7..."
3599,3963,None,0.999929,1976,3964,None,0.999864,1977,"POLYGON ((294364.801 7515343.054, 294364.802 7..."
3600,4160,None,0.598928,2015,3964,None,0.999864,1977,"POLYGON ((294363.504 7515339.030, 294363.690 7..."


In [150]:
delete_lines = []

for i in over['id_1']:
    
    intersec = over[over['id_1']==i]
    
    if intersec.loc[intersec['geometry'].area >= 0.6 * pols.loc[i, 'geometry'].area].any(axis=None) or \
    (3.5 * pols.loc[i, 'geometry'].area < pols.loc[intersec['id_2'], 'geometry'].area).any():
        delete_lines.append(i)
        
delete_lines = (list(set(delete_lines)))
delete_lines.sort()            

In [151]:
pols.loc[delete_lines].to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_delete.geojson'), driver='GeoJSON')

In [ ]:
pols.drop(labels=delete_lines, axis=0, inplace=True)
pols

In [ ]:
os.path.dirname(shape_path_aux)

In [ ]:
pols.buffer(-0.1).buffer(0.1).simplify(0.03).to_file(os.path.join(os.path.dirname(shape_path_aux),'canopy_detection_result_faster_simplify.geojson'), driver='GeoJSON')